In [ ]:
created on: Fri Jan 10 11:06:30 2020
Group 7
@author: C.L.

<h1>Groupe 7 : Images sociales<span class="tocSkip"></span>
    
 Scraping
 
 

#Introduction

In order to train a model able to recognize airplaines type from the outside, we needed training images.
We scrapped two websites in order to train our models for aircraft exteriors. 
For each one, the web links were extracted in order to create a spreadsheet.
Then, we used these dataframes to extract and store these images.

#Environment

#Librairies

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import urllib
import urllib.request
import requests
import os
import errno

import pandas as pd
import numpy as np

import time
from datetime import datetime, timedelta

import io
from PIL import Image

import csv

In [ ]:
# Paths
project_path = './../'
scrap_path = project_path + 'Scraping/'
airliners_csv_path = scrap_path + 'Airliners/'
airliners_path = airliners_csv_path + 'data/'
google_path = scrap_path + 'Google_img/'

# Airliners

In [ ]:
# Boeing : 28482 pages found
# Airbus : 17453 pages found

## Extracting the data

In [ ]:
def create_directories(path: str):
    os.makedirs(path + 'Airbus', exist_ok = True)   
    os.makedirs(path + 'Boeing', exist_ok = True)
    
def get_photos_name_page(link: str):
    """
    For a given Airliner page with the given link, create a dataframe containing for each image found :
    - the link towards the image
    - the manufacturer
    - the aircraft type 
    - a more precise designation
    """
    req = requests.get(link)
    soup = BeautifulSoup(req.text, "lxml")
    AirCrafts = soup.find_all("div", {"class":"ps-v2-results-row"})
    list_DF = []
    for air in AirCrafts:
        try :
            photo = air.find_all("img")[0].attrs['src']
            details_cell = air.find_all("div", {"class" : "ps-v2-results-col ps-v2-results-col-aircraft"})[0]
            details_cell = details_cell.find_all("div", {'class' : "ps-v2-results-col-content"})[0]
            details_cell = details_cell.find_all("div", {"class":"ps-v2-results-col-content-primary"})[0]
            name_cell = details_cell.find_all("div", {"class":"ps-v2-results-display-detail-no-wrapping"})[1]
            name = name_cell.a.text.strip().replace('/', '-')
            if name.split()[0] in ['Airbus', 'Boeing']:
                list_DF.append([photo, name.split()[0], name.split()[1].split('-')[0], name])
        except :
            pass
    DF = pd.DataFrame(list_DF, columns = ['Photo', 'Manufacturer', 'Aircraft_type', 'Designation'])
    return(DF)


def create_df_manufacturer(a : int ,b : int ,manuf_name : str):
    """
    For a given manufacturer, create a dataframe in which each line contains informations related to an aircraft 
    image.
    Parameters : 
    a : page number where the search begins
    b : page number where the search ends
    """
    l_df = []
    dic_code_manuf = {'Airbus' : '2', 'Boeing' : '7'}
    for i in range(a,b):
        print(i)
        link = 'https://www.airliners.net/search?aircraftManufacturer=' + dic_code_manuf[manuf_name]+ \
               '&photoCategory=23&sortBy=dateAccepted&sortOrder=desc&perPage=36&display=detail&page=' + str(i)
        l_df.append(get_photos_name_page(link))
    df = pd.concat(l_df, ignore_index = True)
    return df


In [ ]:
create_directories(airliners_path)

In [ ]:
# Test

df_airbus_test = create_df_manufacturer(1,3,'Airbus')
df_airbus_test.to_csv(scrap_path + 'Airbus_test.csv', sep = ';', index = False)
df_airbus_test.head()

In [ ]:
# Scraping for Airbus

df_boeing_airliners = create_df_manufacturer(1,17453,'Airbus')
df_boeing_airliners.to_csv(scrap_path + 'Airbus_Airliners.csv', sep = ';', index = False)
len(df_airbus_airliners)

In [ ]:
# Scraping for Boeing

df_boeing_airliners = create_df_manufacturer(1,28482,'Boeing')
df_boeing_airliners.to_csv(scrap_path + 'Boeing_Airliners.csv', sep = ';', index = False)
len(df_airbus_airliners)

In [ ]:
## Creating the images from the dataframe

In [ ]:
df_airbus = pd.read_csv(scrap_path + 'Airbus_Airliners.csv', sep = ';')
df_boeing = pd.read_csv(scrap_path + 'Boeing_Airliners.csv', sep = ';')

In [ ]:
# Test
df_airbus = pd.read_csv(scrap_path + 'Airbus_test.csv', sep = ';')


In [ ]:
dic_type = {'Airbus' : ['A220', 'A300B4', 'A310', 'A318', 'A319', 'A320', 'A321', 'A330', 'A340', 'A350', 'A380'],
            'Boeing' : ['717', '727', '737', '747', '757', '767', '777', '787']}

## Creation and storage of the images

In [ ]:
def create_df_passengers(df, manufacturer, dic_type):
    """Create a new dataframe including only rows matching with commercial aircraft and adding a column
    similar to the index.
    
    Parameters : 
        df (type dataframe): dataframe on which the selection must be proceed
        manufacturer (type string): company name ('Airbus' or 'Boeing')
        dic_type (type dictionary): keys are companies names and values are lists of the commercial aircraft 
        of the manufacturer 
    
    Out :
        df_passengers (type dataframe): modified dataframe  
    """
        
    df_passengers = df[df['Aircraft_type'].isin(dic_type[manufacturer])]
    return df_passengers.assign(Number = df_passengers.index)


def get_number_pic_per_type(df, nb, a):
    # df : dataframe with only aircraft types of interest
    # nb : number of pictures to create for each aircraft type
    ix = []
    for el in df['Aircraft_type'].unique().tolist():
        df_el = df[df['Aircraft_type']== el].copy()
        df_el.reset_index(drop = True, inplace = True)
        df_el = df_el.loc[a : a + min(nb-1, len(df_el))]
        ix.extend(df_el['Number'].tolist())
    df_nb = df[df['Number'].isin(ix)]
    return df_nb


def get_from_link_and_write(df, manufacturer, path):
    
    for el in df['Aircraft_type'].unique().tolist():
        df_type = df[df['Aircraft_type']== el]
        fold_name = path + manufacturer + '/' + el
        os.makedirs(fold_name, exist_ok = True)
        my_rows = zip(df_type['Number'], df_type['Photo'])
        for (num, photo) in my_rows:
            img_data = requests.get(photo).content
            try :
                i = Image.open(io.BytesIO(img_data))
                i.save(fold_name + '/' + manufacturer + '_' + str(num) + '_' + el + '.jpg')
            except :
                print(el, num)
                pass       


In [ ]:
df_airbus_passengers = create_df_passengers(df_airbus, 'Airbus', dic_type)
df_boeing_passengers = create_df_passengers(df_boeing, 'Boeing', dic_type)

In [ ]:
df_airbus_passengers

In [ ]:
for el in dic_type['Airbus'] : 
    print(el, len(df_airbus[df_airbus['Aircraft_type']== el]))
    
print('\n')

for el in dic_type['Boeing'] : 
    print(el, len(df_boeing[df_boeing['Aircraft_type']== el]))

In [ ]:
df_airbus_100 = get_number_pic_per_type(df_airbus_passengers, 100, 0)
print(len(df_airbus_100))

In [ ]:
df_airbus_100

In [ ]:
start=time.time()
get_from_link_and_write(df_airbus_100, 'Airbus', airliners_path)
print(time.time()-start)
# 125s needed to write 1100 pictures

In [ ]:
df_boeing_limited = get_number_pic_per_type(df_boeing_passengers, 1000, 0)
get_from_link_and_write(df_boeing_limited, 'Boeing', airliners_path)

# Google images

In [ ]:
driver = webdriver.Chrome("chromedriver.exe")
os.makedirs(google_path, exist_ok = True)

In [ ]:
dic_df_types = {}

In [ ]:
def google_img_by_aircraft_type(aircraft_type, dic_df_types):
    
    """ Update a dictionary with a dataframe giving links to the pictures found on the first page of Google Images
    for a given request.
    
    Parameters :
        aircraft_type (type string) : request for Google Images. Ex : 'Airbus A380'.
        dic_df_types (type dict) : the keys are aircraft_type and the values are dataframes.
    """ 
    
    # Use of &tbs=sur:fc in the link to download only free_to_use images : 
    link = 'https://www.google.com/search?q=' + aircraft_type + "&tbm=isch&tbs=sur:fc"
    df_images = pd.DataFrame(columns=["link", "request", "source", "source_2"])
    driver.get(link)
    time.sleep(5)

    blocks_images = driver.find_elements_by_xpath("//div[contains(@class,'rg_bx rg_di rg_el ivg-i')]")
    for bi in blocks_images:

        site = bi.find_elements_by_xpath(".//span")
        if site : site = site[0].get_attribute("innerHTML")

        source_1 = bi.find_elements_by_xpath(".//img")
        if source_1 : source_1 = source_1[0].get_attribute("data-src")

        source_2 = bi.find_elements_by_xpath("./a")
        if source_2 : source_2 = source_2[0].get_attribute("href")

        row = pd.Series([site, aircraft_type, source_1, source_2], index=df_images.columns)
        df_images = df_images.append(row, ignore_index=True, sort=False)
        
    dic_df_types[aircraft_type] = df_images
    
    
def get_from_source_and_write(request : str, dic_df_types : dict , path : str):
    """
    request : manufacturer name + white space + aircraft-type (ex : 'Airbus A320')
    dic_df_types : 
    path : path to the Google subdirectory
    """
    
    print(request)
    df = dic_df_types[request]
    print(len(df))
    df = df[df['source'].notnull()]
    df.reset_index(inplace = True)
    df = df.assign(Number = df.index)
    print(len(df))
    request = request.split()    
    fold_name = path + request[0] + '/' + request[1]
    os.makedirs(fold_name, exist_ok = True)
    my_rows = zip(df['Number'], df['source'])
    for (num, source) in my_rows:
        img_data = requests.get(source).content
        try :
            i = Image.open(io.BytesIO(img_data))
            i.save(fold_name + '/' + request[0] + '_' + request[1] + '_' + str(num) + '.jpg')
        except :
            pass


In [ ]:
for key in dic_type.keys():
    print(key)
    for aircraft in dic_type[key]:
        print(aircraft)
        google_img_by_aircraft_type(key + ' ' + aircraft, dic_df_types)

In [ ]:
dic_df_types['Boeing 737'].head()

In [ ]:
# Test with an aircraft type 

key = 'Boeing'
#['717', '727', '737', '747', '757', '767', '777', '787']
aircraft = '787'
get_from_source_and_write(key + ' ' + aircraft, dic_df_types, google_path)

In [ ]:
for key in dic_type.keys():
    for aircraft in dic_type[key]:
        get_from_source_and_write(key + ' ' + aircraft, dic_df_types, google_path)   

In [ ]:
### Save dataframes

In [ ]:
for key, val in dic_df_types.items():
    val.to_csv(google_path + "Google_{}.csv".format(str(key)), sep = ';', index = False)
    
# Saving keys to file

with open(google_path + "keys.txt", "w") as f: 
     f.write(str(list(dic_df_types.keys())))

# Reading data from keys
with open(google_path + "keys.txt", "r") as f:
    keys = eval(f.read())

    dic = {}    
    for key in keys:
        dic[key] = pd.read_csv(google_path + "Google_{}.csv".format(str(key)), sep = ';')
    return dic